In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#train = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/train.csv')
#asset_details = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/asset_details.csv')
#example_test = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/example_test.csv')
#example_sample_submission = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/example_sample_submission.csv')
supplemental_train = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/supplemental_train.csv')

#print('train', train.shape)
#print('asset_details', asset_details.shape)
#print('example_test', example_test.shape)
#print('example_sample_submission', example_sample_submission.shape)
print('supplemental_train', supplemental_train.shape)

In [ ]:
def get_hist_data_dict(hist_data):
    """ 
        构建 hist_data_dict ： 
            1. 补足begin_t和end_t之间缺失的数据
            2. 新增day和min字段
    """
    hist_data_dict = {}
    for s in hist_data['Asset_ID'].unique():
        s_hist_data = hist_data[hist_data['Asset_ID']==s]
        begin_t = s_hist_data['timestamp'].values[0]
        end_t = s_hist_data['timestamp'].values[-1]
        complete_ts = [t for t in range(begin_t, end_t+1, 60)]
        complete_close_df = pd.merge(pd.DataFrame(complete_ts, columns=['timestamp']),
                                     s_hist_data[['timestamp', 'Close', 'Target']],
                                     how='left',
                                     on='timestamp'
                                    )
        complete_close_df['Close'][complete_close_df['Close'].isnull()] = 0.5*(complete_close_df['Close'].fillna(method='ffill') + complete_close_df['Close'].fillna(method='bfill'))
        complete_close_df['Target'][complete_close_df['Target'].isnull()] = 0.5*(complete_close_df['Target'].fillna(method='ffill') + complete_close_df['Target'].fillna(method='bfill'))
        complete_close_df['Close'] = complete_close_df['Close'].fillna(method='bfill')
        complete_close_df['Target'] = complete_close_df['Target'].fillna(method='bfill')
        complete_close_df.dropna(subset=['Target'], inplace=True)
        
        complete_close_df['day'] = complete_close_df['timestamp']/60//1440
        complete_close_df['min'] = complete_close_df['timestamp']/60%1440

        hist_data_dict[s] = complete_close_df
    return hist_data_dict

In [ ]:
hist_data_dict = get_hist_data_dict(supplemental_train)

In [ ]:
map_period = 30

close_arr_dict = {s: hist_data_dict[s]['Close'][-map_period:].fillna(method='ffill').values for s in hist_data_dict}
print('close_arr_dict', len(close_arr_dict))

In [ ]:
import gresearch_crypto
import time 

env = gresearch_crypto.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

for (test_df, pred_df) in iter_test:
    for idx, row in enumerate(test_df.iterrows()):
        start = time.time()
        # 解析该行数据
        t, symbol, cnt, o, h, l, c, vol, vwap, rowid = row[1]

        # 构造最新的close_arr，更新close_arr_dict
        close_arr = np.roll(close_arr_dict[symbol], -1)
        close_arr[-1] = c
        close_arr_dict[symbol] = close_arr

        # close_arr进行一阶拟合和二阶拟合
        params_d1 = np.polyfit(np.arange(1, map_period+1), close_arr, 1)
        params_d1 = np.polyfit(np.arange(1, map_period+1), close_arr, 2)
        # 预测pred_c
        pred_t = map_period + 15
        pred_c1 = params_d1[0]*pred_t + params_d1[1]
        pred_c2 = params_d1[0]*pred_t**2 + params_d1[1]*pred_t + params_d1[2]
        pred_c = (pred_c1 + pred_c2)/2     

        # 预测target
        try:
            best_target = np.log(pred_c/c)
        except:
            best_target = 0.0

        # 赋值给pref_df提交
        pred_df.iloc[idx, 1] = best_target        
        print(time.time()-start)
    env.predict(pred_df)   # register your predictions